In [111]:
# bs4
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm

In [112]:
with open('./1.html') as html_file:
    soup = BeautifulSoup(html_file,features="html.parser")
    tables = soup.findAll("div", class_="row mb-2")

entry = []
for table in tables:
    try:
       exists = table.find('h6')
       if exists:
        entry.append(table)
    except:
        del table

In [113]:
len(entry)

20

In [114]:
from bs4 import BeautifulSoup
import datetime, time
from IPython.core.debugger import Pdb
import sys, re
import os.path
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import pandas


PROGS = [
    ("Computer Engineering", "Electrical and Computer Engineering"),
    ("Computer Enginnerin", "Electrical and Computer Engineering"),
    ("Electrical", "Electrical and Computer Engineering"),
    ("ECE", "Electrical and Computer Engineering"),
    ("Computer Sc", "Computer Science"),
    ("Computer  Sc", "Computer Science"),
    ("Computer Sicen", "Computer Science"),
    ("Computer Sien", "Computer Science"),
    ("Computer S Cience", "Computer Science"),
    ("Computer,", "Computer Science"),
    ("Computers,", "Computer Science"),
    ("ComputerScience", "Computer Science"),
    ("Human Computer Interaction", "Human Computer Interaction"),
    ("Human-Computer Interaction", "Human Computer Interaction"),
    ("Human-computer Interaction", "Human Computer Interaction"),
    ("software engineering", "Software Engineering"),
    ("Embedded", "Electrical and Computer Engineering"),
    ("Computer Eng", "Electrical and Computer Engineering"),
    ("Computer Vision", "Computer Science"),
]

# ('computer graphics', 'Game Development'),
# ('computer gam', 'Game Development'),
# ('Computer Systems', 'Computer Systems Engineering'),
# ('Computer And Systems', 'Computer Systems Engineering'),
# ('Computer & Systems', 'Computer Systems Engineering'),
# ('Information Technology', 'IT'),
# ('Communication', 'Computers and Communication'),
# ('Computer Network', 'Computer Networking'),
# ('Computer And Computational Sciences', 'Computer And Computational Sciences'),
# ('Computer Music', 'Computer Music'),
# ('Computer Control And Automation', 'Computer Control And Automation'),
# ('Computer Aided Mechanical Engineering', 'CAME'),
# ('Computer Art', 'Computer Art'),
# ('Computer Animation', 'Computer Art'),
# ('composition and computer technologies', 'Computer Art'),
# ('computer forensics', 'Computer Art')]

DEGREE = [
    (" MFA", "MFA"),
    (" M Eng", "MEng"),
    (" MEng", "MEng"),
    (" M.Eng", "MEng"),
    (" Masters", "MS"),
    (" PhD", "PhD"),
    (" MBA", "MBA"),
    (" Other", "Other"),
    (" EdD", "Other"),
]

STATUS = {
    "A": "American",
    "U": "International with US Degree",
    "I": "International",
    "O": "Other",
}

COLLEGES = [
    ("Stanford", "Stanford"),
    ("MIT", "MIT"),
    ("CMU", "CMU"),
    ("Cornell", "Cornell"),
]

errlog = {"major": [], "gpa": [], "general": [], "subject": []}


def process(index, col):
    global err
    inst, major, degree, season, status, date_add, date_add_ts, comment = (
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
    )

    if len(col) != 6:
        return []
    try:
        inst = col[0].text.strip()
    except:
        print("Couldn't retrieve institution")
    try:
        major = None
        progtext = col[1].text.strip()

        if not "," in progtext:
            print("no comma")
            errlog["major"].append((index, col))
        else:
            parts = progtext.split(",")
            major = parts[0].strip()
        progtext = " ".join(parts[1:])
        for p, nam in PROGS:
            if p.lower() in major.lower():
                major = nam
                break

        degree = None
        for (d, deg) in DEGREE:
            if d in progtext:
                degree = deg
                break
        if not degree:
            degree = "Other"

        season = None
        mat = re.search("\([SF][012][0-9]\)", progtext)
        if mat:
            season = mat.group()[1:-1]
        else:
            mat = re.search("\(\?\)", progtext)
            if mat:
                season = None
    except NameError as e:
        print(e)
    except:
        print("Unexpected error:", sys.exc_info()[0])
    try:
        extra = col[2].find(class_="extinfo")
        gpafin, grev, grem, grew, new_gre, sub = None, None, None, None, None, None
        if extra:
            gre_text = extra.text.strip()
            gpa = re.search("Undergrad GPA: ((?:[0-9]\.[0-9]{1,2})|(?:n/a))", gre_text)
            general = re.search(
                "GRE General \(V/Q/W\): ((?:1[0-9]{2}/1[0-9]{2}/(?:(?:[0-6]\.[0-9]{2})|(?:99\.99)|(?:56\.00)))|(?:n/a))",
                gre_text,
            )
            new_gref = True
            subject = re.search("GRE Subject: ((?:[2-9][0-9]0)|(?:n/a))", gre_text)

            if gpa:
                gpa = gpa.groups(1)[0]
                if not gpa == "n/a":
                    try:
                        gpafin = float(gpa)
                    except:
                        print("Couldn't convert gpa to float")
            else:
                errlog["gpa"].append((index, gre_text))
            if not general:
                general = re.search(
                    "GRE General \(V/Q/W\): ((?:[2-8][0-9]0/[2-8][0-9]0/(?:(?:[0-6]\.[0-9]{2})|(?:99\.99)|(?:56\.00)))|(?:n/a))",
                    gre_text,
                )
                new_gref = False

            if general:
                general = general.groups(1)[0]
                if not general == "n/a":
                    try:
                        greparts = general.split("/")
                        if (
                            greparts[2] == "99.99"
                            or greparts[2] == "0.00"
                            or greparts[2] == "56.00"
                        ):
                            grew = None
                        else:
                            grew = float(greparts[2])
                        grev = int(greparts[0])
                        grem = int(greparts[1])
                        new_gre = new_gref
                        if new_gref and (
                            grev > 170
                            or grev < 130
                            or grem > 170
                            or grem < 130
                            or (grew and (grew < 0 or grew > 6))
                        ):
                            errlog["general"].append((index, gre_text))
                            grew, grem, grev, new_gre = None, None, None, None
                        elif not new_gref and (
                            grev > 800
                            or grev < 200
                            or grem > 800
                            or grem < 200
                            or (grew and (grew < 0 or grew > 6))
                        ):
                            errlog["general"].append((index, gre_text))
                            grew, grem, grev, new_gre = None, None, None, None
                    except Exception as e:
                        print(e)
            else:
                errlog["general"].append((index, gre_text))

            if subject:
                subject = subject.groups(1)[0]
                if not subject == "n/a":
                    sub = int(subject)
            else:
                errlog["subject"].append((index, gre_text))

            extra.extract()
        decision = col[2].text.strip()
        try:
            decisionfin, method, decdate, decdate_ts = None, None, None, None
            (decisionfin, method, decdate) = re.search(
                "((?:Accepted)|(?:Rejected)|(?:Wait listed)|(?:Other)|(?:Interview))? ?via ?((?:E-[mM]ail)|(?:Website)|(?:Phone)|(?:Other)|(?:Postal Service)|(?:POST)|(?:Unknown))? ?on ?([0-9]{1,2} [A-Z][a-z]{2} [0-9]{4})?",
                decision,
            ).groups()
            if method and method == "E-Mail":
                method = "E-mail"
            if method and method == "Unknown":
                method = "Other"
            if method and method == "POST":
                method = "Postal Service"
            if decdate:
                try:
                    decdate_date = datetime.datetime.strptime(decdate, "%d %b %Y")
                    decdate_ts = decdate_date.strftime("%s")
                    decdate = decdate_date.strftime("%d-%m-%Y")
                except Exception as e:
                    decdate_date, decdate_ts, decdate = None, None, None
        except Exception as e:
            print("Couldn't assign method of reporting")
    except Exception as e:
        print("Extra information error")
    try:
        statustxt = col[3].text.strip()
        if statustxt in STATUS:
            status = STATUS[statustxt]
        else:
            status = None
    except:
        print("Couldn't retrieve status")
    try:
        date_addtxt = col[4].text.strip()
        date_add_date = datetime.datetime.strptime(date_addtxt, "%d %b %Y")
        date_add_ts = date_add_date.strftime("%s")
        date_add = date_add_date.strftime("%d-%m-%Y")
    except:
        print("Couldn't retrieve date_add")
    try:
        comment = col[5].text.strip()
    except:
        print("Couldn't retrieve the comment")
    res = [
        inst,
        major,
        degree,
        season,
        decisionfin,
        method,
        decdate,
        decdate_ts,
        gpafin,
        grev,
        grem,
        grew,
        new_gre,
        sub,
        status,
        date_add,
        date_add_ts,
        comment,
    ]
    return res


In [115]:
def prettier_strings(string):
    try:
        string = string.replace("\t", "") \
            .replace("\r", "") \
            .replace("\n", "") \
            .replace("\xa0", " ")
    except:
        print("Couldn't prettify the string")
    
    return string

In [156]:
def parse_details(details):
    status = None
    try:
        author_comment = prettier_strings(details[0])
        if 'accepted' in author_comment.lower():
            status = 'Accepted'

    except:
        author_comment = None
        
    try:
        if status is None:
            status = prettier_strings(details[1]).split('on')[0].strip()

    except:
        status = None

    try: 
        descison_date = prettier_strings(details[1]).split('on')[1].strip()
    except:
        descison_date = None
    try:
        season = prettier_strings(details[2]).split()[0].strip()
        year = prettier_strings(details[2]).split()[1].strip()
    except:
        season, year = None, None
    try:    
        international = prettier_strings(details[3]).strip()
    except:
        international = None

    other_details = {}
    try:
        for detail in details[4:]:
            detail = prettier_strings(detail)
            if len(detail.split()) > 1:
                value = prettier_strings(detail).split()[-1]
                value = int(value) if value.isdigit() else value
                key = ''.join(prettier_strings(detail).split()[:-1])

                other_details[key] = value
            elif detail != '':
                other_details['level'] = detail.strip()
    except Exception as e:
        print('loop',e)

    # find season
    for d in details:
        if 'Fall' in d and len(d.split()) == 2:
            season = 'Fall'
            year = d.split()[1]
        elif 'Spring' in d and len(d.split()) == 2:
            season = 'Spring'
            year = d.split()[1]




    return { 
        'author_comment': author_comment, 
        'status': status, 
        'descison_date': descison_date, 
        'season': season, 'year': year, 
        'international': international,
        **other_details,

    }

In [157]:
def parse(entry):
    """
    Return:
    -------
    inst, major, degree, season, status, date_add, date_add_ts, comment = (
        None,
        None,
        None,
        None,
        None,
        None,
        None,
        None,
    )
    """
    try:
        title = entry.find('h6', class_ = 'mt-3 fw-normal').text
        comment = entry.find('span', class_ = 'text-secondary ms-3').text
        # remove the comment from the title
        title = title.replace(comment, '').strip()
        program = title.split(',')[0].strip()
        university = ''.join(title.split(',')[1:])
        program = prettier_strings(program)
        university = prettier_strings(university)
        spans = entry.findAll('span')
        details = [span.text for span in spans if span.text]
        parsed_details = parse_details(details)
        
        return {
            'title': title,
            'program': program,
            'university': university,
            **parsed_details
        }
    except Exception as e:
        print(e)
        return None

In [158]:
def new_process(entries : list):
    """
    Process the entries and return a list of dictionaries
    """
    res = []
    for entry in entries:
        parsed = parse(entry)
        if parsed:
            res.append(parsed)
    return res

In [159]:
df = pd.DataFrame()
NUM_SCRAPED = 257

In [160]:
adf = pd.DataFrame()
NUM_SCRAPED = 257

for i in tqdm(range(1, NUM_SCRAPED + 1)):
    with open(f'./{i}.html', 'r') as f:
            soup = BeautifulSoup(f,features="html.parser")
    tables = soup.findAll("div", class_="row mb-2")

    entry = []
    for table in tables:
        try:
            exists = table.find('h6')
            if exists:
                entry.append(table)
        except:
            del table

    data = new_process(entry)
    df = df.append(data, ignore_index=True)

  0%|          | 0/257 [00:00<?, ?it/s]

In [161]:
df

,title,program,university,author_comment,status,descison_date,season,year,international,GRE,GREV,GREAW,GPA,level
0,"Computer Science, University Of Massachusetts,...",Computer Science,University Of Massachusetts Amherst,List of Acceptances for Spring 2023 :USC (MSCS...,Accepted,20 Oct,Fall,2022,International,167.0,157.0,3.50,3.96,Masters
1,"Materials Science And Engineering, University ...",Materials Science And Engineering,University Of Pennsylvania,Actually this was something long long time ago...,Accepted,28 Feb,Fall,2022,International,153.0,168.0,4.50,3.80,PhD
2,"Computer Science, University Of Illinois Urban...",Computer Science,University Of Illinois Urbana-Champaign,Acceptedon 4 Nov,Accepted,None,Fall,2022,Masters,NaN,NaN,NaN,NaN,NaN
3,"Computer Science, University Of Illinois Urban...",Computer Science,University Of Illinois Urbana-Champaign,Hey I am Prathamesh and currently Iâ€™m pursui...,Accepted,4 Nov,Fall,2022,International,169.0,160.0,4.00,NaN,Masters
4,"Computer Science, University Of Massachusetts,...",Computer Science,University Of Massachusetts Amherst,Spring 2023; Disappointed; Frustrated with the...,Rejected,31 Oct,Fall,2022,International,167.0,157.0,4.00,3.22,Masters
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5135,"Computer Science, Virginia Tech",Computer Science,Virginia Tech,"For Spring 23, MEngg in CS at Blacksburg Campus",Accepted,17 Oct,Fall,2022,International,NaN,NaN,NaN,NaN,Masters
5136,"Mechanical Engineering, Birla Institute Of Tec...",Mechanical Engineering,Birla Institute Of Technology And Science Pilani,Wait listed,Fall 2022,None,Fall,2022,Other,NaN,NaN,NaN,NaN,NaN
5137,"Computer Science, Massachusetts Institute Of T...",Computer Science,Massachusetts Institute Of Technology,hello pepul,Accepted,13 Oct,Spring,2021,American,168.0,168.0,6.50,4.00,PhD
5138,"Communication Sciences And Disorders, New York...",Communication Sciences And Disorders,New York University,I received an email stating to sign something ...,Accepted,12 Oct,Fall,2022,American,NaN,NaN,NaN,3.77,Masters


In [108]:
result_urbana = df[df['university'].str.contains('Urbana')]

In [162]:
df.to_csv('result.csv', index=False)